In [1]:
import numpy as np

# --- Parâmetros da Simulação ---
P0 = 50.0       # Preço inicial da ação em R$
mu = 0.10       # Drift (taxa de retorno esperada)
sigma = 0.25    # Volatilidade
T = 1.0         # Horizonte de tempo em anos
N = 252         # Número de passos no tempo (dias úteis em um ano)
dt = T / N      # Intervalo de tempo
num_simulations = 100000  # Número de simulações de Monte Carlo

# --- Simulação das Trajetórias de Preços ---

# Para garantir que os resultados sejam reprodutíveis
np.random.seed(42)

# Gera todos os movimentos aleatórios de uma vez para eficiência
# Dimensões: (número de simulações, número de passos)
random_shocks = np.random.standard_normal((num_simulations, N))

# Inicializa uma matriz para armazenar as trajetórias dos preços
# Começa com o preço inicial P0 em todas as simulações
price_paths = np.zeros((num_simulations, N + 1))
price_paths[:, 0] = P0

# Calcula o preço para cada passo de tempo em todas as simulações
for t in range(1, N + 1):
    # Fórmula do Movimento Geométrico Browniano (GBM)
    price_paths[:, t] = price_paths[:, t - 1] * np.exp((mu - 0.5 * sigma**2) * dt +
                                                       sigma * np.sqrt(dt) * random_shocks[:, t - 1])

# --- Cálculo das Probabilidades ---

# 1) Probabilidade de o preço da ação estar acima de R$ 70 após um ano
final_prices = price_paths[:, -1]
prob_1 = np.sum(final_prices > 70) / num_simulations

# 2) Probabilidade de o preço da ação estar acima de R$ 70 em algum momento do próximo ano
max_prices = np.max(price_paths, axis=1)
prob_2 = np.sum(max_prices > 70) / num_simulations

# 3) Probabilidade de o preço estar acima de R$ 70 no final do ano E ter caído abaixo de R$ 50 em algum momento
min_prices = np.min(price_paths, axis=1)
condition_3 = (final_prices > 70) & (min_prices < 50)
prob_3 = np.sum(condition_3) / num_simulations

# 4) Probabilidade de o preço estar acima de R$ 70 no final do ano E abaixo de R$ 50 nos primeiros seis meses
# Metade dos passos corresponde a seis meses (N/2)
first_six_months_paths = price_paths[:, :N//2 + 1]
min_first_six_months = np.min(first_six_months_paths, axis=1)
condition_4 = (final_prices > 70) & (min_first_six_months < 50)
prob_4 = np.sum(condition_4) / num_simulations

# 5) Probabilidade de o preço cruzar o limiar de R$ 70 (de baixo para cima) três vezes em um ano
crossings_count = np.zeros(num_simulations)
threshold = 70
for i in range(num_simulations):
    # Verifica quando o preço cruza o limiar de R$ 70 vindo de baixo
    # (P_t-1 < 70) e (P_t >= 70)
    crossings = np.sum((price_paths[i, :-1] < threshold) & (price_paths[i, 1:] >= threshold))
    if crossings == 3:
        crossings_count[i] = 1

prob_5 = np.sum(crossings_count) / num_simulations

# --- Exibição dos Resultados ---
print("Resultados da Simulação de Monte Carlo")
print("-" * 40)
print(f"Número de Simulações: {num_simulations}")
print("-" * 40)
print(f"1) Probabilidade do preço > R$ 70 em um ano: {prob_1:.2%}")
print(f"2) Probabilidade do preço > R$ 70 em algum momento no ano: {prob_2:.2%}")
print(f"3) Probabilidade do preço > R$ 70 no final E < R$ 50 em algum momento: {prob_3:.2%}")
print(f"4) Probabilidade do preço > R$ 70 no final E < R$ 50 nos primeiros 6 meses: {prob_4:.2%}")
print(f"5) Probabilidade de cruzar R$ 70 (de baixo para cima) 3 vezes: {prob_5:.4%}")

Resultados da Simulação de Monte Carlo
----------------------------------------
Número de Simulações: 100000
----------------------------------------
1) Probabilidade do preço > R$ 70 em um ano: 14.07%
2) Probabilidade do preço > R$ 70 em algum momento no ano: 23.70%
3) Probabilidade do preço > R$ 70 no final E < R$ 50 em algum momento: 11.81%
4) Probabilidade do preço > R$ 70 no final E < R$ 50 nos primeiros 6 meses: 11.80%
5) Probabilidade de cruzar R$ 70 (de baixo para cima) 3 vezes: 3.7600%
